In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
main_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

In [3]:
data = requests.get(main_url)

In [4]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
data = requests.get(team_urls[0])

In [ ]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [ ]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [ ]:
shooting.head()


In [ ]:
shooting.columns = shooting.columns.droplevel()

In [ ]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [ ]:
team_data.head()

In [39]:
years = list(range(2024, 2014, -1))
all_matches = []

In [40]:
main_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

In [41]:
for year in years:
    data = requests.get(main_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    main_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "La Liga"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(5)

C:\Users\Pedro\AppData\Local\Temp\ipykernel_16168\1312788222.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Pedro\AppData\Local\Temp\ipykernel_16168\1312788222.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
C:\Users\Pedro\AppData\Local\Temp\ipykernel_16168\1312788222.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
C:\Users\Pedro\AppData\Local\Temp\ipykernel_16168\1312788222.py:21: FutureWarning: Passing literal 

KeyError: "['FK'] not in index"

In [42]:
len(all_matches)

140

In [43]:
match_df = pd.concat(all_matches)

In [44]:
match_df.columns = [c.lower() for c in match_df.columns]

In [45]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-12,21:30,La Liga,Matchweek 1,Sat,Away,W,2.0,0.0,Athletic Club,...,Match Report,NaN,14.0,8.0,16.0,2.0,0,0,2024,Real Madrid
1,2023-08-19,19:30,La Liga,Matchweek 2,Sat,Away,W,3.0,1.0,Almería,...,Match Report,NaN,25.0,9.0,17.0,1.0,0,0,2024,Real Madrid
2,2023-08-25,21:30,La Liga,Matchweek 3,Fri,Away,W,1.0,0.0,Celta Vigo,...,Match Report,NaN,9.0,2.0,19.4,0.0,0,1,2024,Real Madrid
3,2023-09-02,16:15,La Liga,Matchweek 4,Sat,Home,W,2.0,1.0,Getafe,...,Match Report,NaN,26.0,12.0,17.7,0.0,0,0,2024,Real Madrid
4,2023-09-17,21:00,La Liga,Matchweek 5,Sun,Home,W,2.0,1.0,Real Sociedad,...,Match Report,NaN,17.0,8.0,15.9,1.0,0,0,2024,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2018-04-22,16:15,La Liga,Matchweek 34,Sun,Home,W,2,0,Real Sociedad,...,Match Report,NaN,5.0,4.0,11.5,0.0,0,0,2018,Malaga
36,2018-04-30,21:00,La Liga,Matchweek 35,Mon,Away,L,1,2,Betis,...,Match Report,NaN,9.0,3.0,16.7,0.0,0,0,2018,Malaga
37,2018-05-06,12:00,La Liga,Matchweek 36,Sun,Home,L,0,3,Alavés,...,Match Report,NaN,16.0,4.0,15.4,1.0,0,0,2018,Malaga
38,2018-05-13,16:15,La Liga,Matchweek 37,Sun,Away,L,1,4,Espanyol,...,Match Report,NaN,8.0,2.0,17.3,0.0,1,1,2018,Malaga


In [46]:
match_df.to_csv("matches.csv")

In [15]:
match_df.season.value_counts()

season
2023    760
2022    760
2021    760
2020    760
2019    760
2018    760
2024    596
Name: count, dtype: int64

In [17]:
38*20

760

In [23]:
match_df[(match_df["round"] == "Matchweek 1") & (match_df["season"] == 2024)]

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,formation,referee,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-12,21:30,Matchweek 1,Sat,Away,W,2.0,0.0,Athletic Club,0.9,...,4-1-2-1-2◆,Jesús Gil,14.0,8.0,16.0,2.0,0,0,2024,Real Madrid
0,2023-08-13,21:30,Matchweek 1,Sun,Away,D,0.0,0.0,Getafe,1.4,...,4-2-3-1,César Soto,14.0,3.0,19.9,1.0,0,0,2024,Barcelona
0,2023-08-12,17:00,Matchweek 1,Sat,Away,D,1.0,1.0,Real Sociedad,0.8,...,4-1-4-1,Francisco Hernández,6.0,1.0,12.2,0.0,0,0,2024,Girona
0,2023-08-12,21:30,Matchweek 1,Sat,Home,L,0.0,2.0,Real Madrid,0.4,...,4-2-3-1,Jesús Gil,7.0,1.0,15.3,0.0,0,0,2024,Athletic Club
0,2023-08-14,21:30,Matchweek 1,Mon,Home,W,3,1,Granada,2.4,...,3-5-2,Juan Pulido,14.0,6.0,14.4,0.0,0,0,2024,Atletico Madrid
0,2023-08-12,17:00,Matchweek 1,Sat,Home,D,1,1,Girona,1.0,...,4-3-3,Francisco Hernández,15.0,2.0,20.0,1.0,0,0,2024,Real Sociedad
0,2023-08-13,19:30,Matchweek 1,Sun,Away,W,2.0,1.0,Villarreal,1.1,...,4-2-3-1,Miguel Ángel Ortiz Arias,12.0,5.0,18.6,1.0,0,0,2024,Real Betis
0,2023-08-11,22:00,Matchweek 1,Fri,Away,W,2.0,1.0,Sevilla,1.1,...,4-4-2,José Sánchez,10.0,3.0,16.0,1.0,0,0,2024,Valencia
0,2023-08-13,17:00,Matchweek 1,Sun,Away,W,2.0,0.0,Celta Vigo,1.0,...,4-3-3,Pablo González,11.0,5.0,15.7,1.0,0,0,2024,Osasuna
0,2023-08-13,19:30,Matchweek 1,Sun,Home,L,1,2,Betis,0.9,...,4-3-3,Miguel Ángel Ortiz Arias,18.0,5.0,16.6,0.0,0,0,2024,Villarreal


In [83]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5156 entries, 0 to 39
Data columns (total 23 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        5156 non-null   datetime64[ns]
 1   round       5156 non-null   int64         
 2   day         5156 non-null   int32         
 3   venue       5156 non-null   int64         
 4   gf          5156 non-null   float64       
 5   ga          5156 non-null   float64       
 6   opponent    5156 non-null   int64         
 7   xg          5156 non-null   float64       
 8   xga         5156 non-null   float64       
 9   poss        5156 non-null   float64       
 10  attendance  4180 non-null   float64       
 11  formation   5156 non-null   object        
 12  referee     5156 non-null   object        
 13  sh          5156 non-null   float64       
 14  sot         5156 non-null   float64       
 15  dist        5154 non-null   float64       
 16  fk          5156 non-null   flo

In [21]:
5156/2


2578.0

In [47]:
match_df.drop(columns=["captain", "notes", "match report", "comp"], inplace=True)

In [12]:
set(match_df.opponent.unique()) - set(match_df.team.unique())

set()

In [13]:
set(match_df.team.unique()) - set(match_df.opponent.unique())

set()

In [25]:
match_df.team.unique()

array(['Real Madrid', 'Barcelona', 'Girona', 'Athletic Club',
       'Atletico Madrid', 'Real Sociedad', 'Real Betis', 'Valencia',
       'Osasuna', 'Villarreal', 'Getafe', 'Las Palmas', 'Alaves',
       'Sevilla', 'Mallorca', 'Rayo Vallecano', 'Celta Vigo', 'Cadiz',
       'Granada', 'Almeria', 'Valladolid', 'Espanyol', 'Elche', 'Levante',
       'Huesca', 'Eibar', 'Leganes', 'Deportivo La Coruna', 'Malaga'],
      dtype=object)

In [36]:
match_df = pd.read_csv("matches.csv")

In [48]:
team_correction = {'Alavés': 'Alaves', 'Almería': 'Almeria', 'Atlético Madrid': 'Atletico Madrid', 'Betis': 'Real Betis', 'Cádiz': 'Cadiz', 'La Coruña': 'Deportivo La Coruna', 'Leganés': 'Leganes', 'Málaga': 'Malaga'}


In [49]:
match_df['opponent'] = match_df['opponent'].replace(team_correction)

In [51]:
match_df

,date,time,round,day,venue,result,gf,ga,opponent,xg,...,formation,referee,sh,sot,dist,fk,pk,pkatt,season,team
0,2023-08-12,21:30,1,Sat,Away,W,2.0,0.0,Athletic Club,0.9,...,4-1-2-1-2◆,Jesús Gil,14.0,8.0,16.0,2.0,0,0,2024,Real Madrid
1,2023-08-19,19:30,2,Sat,Away,W,3.0,1.0,Almeria,2.0,...,4-3-1-2,José Sánchez,25.0,9.0,17.0,1.0,0,0,2024,Real Madrid
2,2023-08-25,21:30,3,Fri,Away,W,1.0,0.0,Celta Vigo,1.4,...,4-1-2-1-2◆,Isidro Díaz de Mera,9.0,2.0,19.4,0.0,0,1,2024,Real Madrid
3,2023-09-02,16:15,4,Sat,Home,W,2.0,1.0,Getafe,2.8,...,4-3-1-2,Mario Melero,26.0,12.0,17.7,0.0,0,0,2024,Real Madrid
4,2023-09-17,21:00,5,Sun,Home,W,2.0,1.0,Real Sociedad,2.0,...,4-1-2-1-2◆,César Soto,17.0,8.0,15.9,1.0,0,0,2024,Real Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,2018-04-22,16:15,34,Sun,Home,W,2,0,Real Sociedad,0.9,...,4-4-1-1,Javier Estrada,5.0,4.0,11.5,0.0,0,0,2018,Malaga
36,2018-04-30,21:00,35,Mon,Away,L,1,2,Real Betis,0.8,...,3-4-3,Carlos del Cerro,9.0,3.0,16.7,0.0,0,0,2018,Malaga
37,2018-05-06,12:00,36,Sun,Home,L,0,3,Alaves,2.0,...,4-4-2,José González,16.0,4.0,15.4,1.0,0,0,2018,Malaga
38,2018-05-13,16:15,37,Sun,Away,L,1,4,Espanyol,1.9,...,3-4-3,Juan Martínez,8.0,2.0,17.3,0.0,1,1,2018,Malaga


In [50]:
match_df['round'] = match_df['round'].str.extract('(\d+)')
match_df['round'] = pd.to_numeric(match_df['round'])

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Pedro\AppData\Local\Temp\ipykernel_16168\691670308.py:1: SyntaxWarning: invalid escape sequence '\d'
  match_df['round'] = match_df['round'].str.extract('(\d+)')


In [6]:
match_df.set_index('Unnamed: 0', inplace=True)

In [52]:
match_df['date'] = pd.to_datetime(match_df['date'])

In [ ]:
match_df[(match_df["season"] == 20

In [8]:
match_df['time'] = pd.to_datetime(match_df['time']).dt.hour
match_df['time_group'] = pd.cut(match_df['time'], bins=[0, 16, 20, 24], labels=[0, 1, 2], right=False)

C:\Users\Pedro\AppData\Local\Temp\ipykernel_16168\2982577516.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  match_df['time'] = pd.to_datetime(match_df['time']).dt.hour


In [42]:
match_df.venue.value_counts()

venue
1    2578
0    2578
Name: count, dtype: int64

In [9]:
match_df.drop(columns=['time'], inplace=True)

In [53]:
match_df["day"] = match_df["date"].dt.dayofweek

In [54]:
map_venue = { "Home" : 0, "Away": 1 }

In [55]:
match_df["venue"] = match_df["venue"].replace(map_venue)

In [56]:
map_result = { "W": 0, "L": 1, "D":2}

In [57]:
match_df["target"] = match_df["result"].replace(map_result)


In [58]:
match_df.drop(columns=['result'], inplace=True)

In [51]:
match_df["season"].nunique()

7

In [16]:
teams = match_df["opponent"].unique()

In [17]:
team_dict = {}

In [59]:
team_dict = {team: index for index, team in enumerate(teams)}

In [60]:
team_dict

{'Athletic Club': 0,
 'Almeria': 1,
 'Celta Vigo': 2,
 'Getafe': 3,
 'Real Sociedad': 4,
 'Atletico Madrid': 5,
 'Las Palmas': 6,
 'Girona': 7,
 'Osasuna': 8,
 'Sevilla': 9,
 'Barcelona': 10,
 'Rayo Vallecano': 11,
 'Valencia': 12,
 'Cadiz': 13,
 'Granada': 14,
 'Real Betis': 15,
 'Villarreal': 16,
 'Alaves': 17,
 'Mallorca': 18,
 'Real Madrid': 19,
 'Valladolid': 20,
 'Elche': 21,
 'Espanyol': 22,
 'Levante': 23,
 'Huesca': 24,
 'Eibar': 25,
 'Leganes': 26,
 'Malaga': 27,
 'Deportivo La Coruna': 28}

In [61]:
match_df["opponent"] = match_df["opponent"].replace(team_dict)

In [62]:
match_df["team"] = match_df["team"].replace(team_dict)

In [65]:
match_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5160 entries, 0 to 39
Data columns (total 22 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       5160 non-null   datetime64[ns]
 1   time       5160 non-null   object        
 2   round      5160 non-null   int64         
 3   day        5160 non-null   int32         
 4   venue      5160 non-null   int64         
 5   gf         5160 non-null   object        
 6   ga         5160 non-null   object        
 7   opponent   5160 non-null   int64         
 8   xg         5160 non-null   float64       
 9   xga        5160 non-null   float64       
 10  poss       5160 non-null   float64       
 11  formation  5160 non-null   object        
 12  referee    5160 non-null   object        
 13  sh         5160 non-null   float64       
 14  sot        5160 non-null   float64       
 15  dist       5158 non-null   float64       
 16  fk         5160 non-null   float64       
 17  pk

In [66]:
match_df.head()

,date,time,round,day,venue,gf,ga,opponent,xg,xga,...,referee,sh,sot,dist,fk,pk,pkatt,season,team,target
0,2023-08-12,21:30,1,5,1,2.0,0.0,0,0.9,0.4,...,Jesús Gil,14.0,8.0,16.0,2.0,0,0,2024,19,0
1,2023-08-19,19:30,2,5,1,3.0,1.0,1,2.0,1.3,...,José Sánchez,25.0,9.0,17.0,1.0,0,0,2024,19,0
2,2023-08-25,21:30,3,4,1,1.0,0.0,2,1.4,1.2,...,Isidro Díaz de Mera,9.0,2.0,19.4,0.0,0,1,2024,19,0
3,2023-09-02,16:15,4,5,0,2.0,1.0,3,2.8,0.4,...,Mario Melero,26.0,12.0,17.7,0.0,0,0,2024,19,0
4,2023-09-17,21:00,5,6,0,2.0,1.0,4,2.0,1.6,...,César Soto,17.0,8.0,15.9,1.0,0,0,2024,19,0


In [26]:
match_df = match_df.rename_axis(None)

In [64]:
match_df.drop(columns=['attendance'], inplace=True)

In [31]:
match_df['time_group'] = pd.to_numeric(match_df['time_group'])

In [67]:
match_df.to_csv("clean_data.csv")